In [63]:
import numpy as np
import random 
import pandas as pd

In [64]:
def post_money_safe(val, inv):
    return inv / val * 100

def pre_money_safe(pre, stake, inv):
    return stake * pre / (pre+inv)


# B brings in 10MM on a 100MM post money valuation on ABC
# how much B owns?

post_val = 100
new_inv = 10

b_post_stake = post_money_safe(post_val, new_inv)
print(b_post_stake)

# Investor A had 10% stake on ABC (10MM investment with a 100MM pre money Valuation)
# B brings in an additional 10MM on a new round
# how much A still have?

a_pre_stake = 50
pre_val = 90

a_post_stake = pre_money_safe(pre_val, a_pre_stake, new_inv)
print(a_post_stake)

10.0
45.0


In [65]:
class Investment:

    def __init__(self, invested, round):
        self.invested = invested
        self.round = round
        self.shares = self.invested / self.round.price

class Investor:

    def __init__(self, name):
        self.name = name
        self.investments = []

    def new_investment(self, invested, round):
        investment = Investment(invested, round)
        self.investments.append(investment)

    def total_shares(self):
        shares = 0
        for i in self.investments:
            shares += i.shares 
        return shares
    
    def total_invested(self):
        invested = 0
        for i in self.investments:
            invested += i.invested 
        return invested

class Covenants:
    
    def __init__(self, covenants={}):
        self.liquidity_preference = covenants.get('liquidity_preference', 0)
           
class Round:

    def __init__(self, name, invested, valuation, shares, covenants):
        self.name = name
        self.invested = invested
        self.valuation = valuation
        self.shares = shares
        self.price = valuation/shares
        self.covenants = covenants

class BookInvestor:
    def __init__(self, name, stake):
        self.name = name
        self.stake = stake / 100

class Deal:

    def __init__(self, investment, stake):
        self.investment = investment
        self.stake = stake / 100
        self.book = []

    def add_investor(self, investor):
        self.book.append(investor)

class Company:
    
    def __init__(self, name, equity, shares):
        self.name = name

        self.incorporation = Round('incorporation', equity, equity, shares, Covenants())

        self.founder = Investor('founder')
        self.founder.new_investment(equity, self.incorporation)

        self.investors = [self.founder]
        self.rounds = [self.incorporation]

    def current_stake(self):
        cap = []
        for i in self.investors:
            last_round = self.rounds[-1]
            total_shares = last_round.shares
            investor_shares = i.total_shares()
            cap.append({
                'investor': i.name,
                'stake': investor_shares / total_shares * 100
            })
        return cap

    def get_investor(self, name):
        for i in self.investors:
            if name == i.name:
                return i
        return None

    def add_investor(self, name):
        new_investor = Investor(name)
        self.investors.append(new_investor)
        return new_investor

    def get_add_investor(self, name):
        investor = self.get_investor(name)
        if investor is None:
            investor = self.add_investor(name)
        return investor

    def new_round(self, name, deal, covenants):

        post_money_valuation = deal.investment / deal.stake
        pre_money_valuation = post_money_valuation -  deal.investment
        
        total_shares =  self.rounds[-1].shares * ( 1 + deal.investment / pre_money_valuation)
        
        round = Round(name, deal.investment, post_money_valuation, total_shares, covenants = covenants)
        
        self.rounds.append(round)

        for i in deal.book:
            new_investor = self.get_add_investor(i.name)
            new_investor.new_investment(deal.investment * i.stake , round)
            
        return None

In [66]:
capital = 1e4
price_per_share = 0.01

company = Company('consorciei', capital, capital/price_per_share)
company.incorporation.valuation

10000.0

In [67]:
cap = company.current_stake()
cap

[{'investor': 'founder', 'stake': 100.0}]

In [68]:
deal = Deal(10000, 10)
deal.add_investor(BookInvestor('ze', 100))
company.new_round('seed', deal, Covenants({}))

deal = Deal(500000, 20)
deal.add_investor(BookInvestor('tera', 80))
deal.add_investor(BookInvestor('patria', 20))
covenants = Covenants({'liquidity_preference': 2})
company.new_round('a', deal, covenants)

In [69]:
company.current_stake()

[{'investor': 'founder', 'stake': 72.0},
 {'investor': 'ze', 'stake': 8.000000000000002},
 {'investor': 'tera', 'stake': 16.0},
 {'investor': 'patria', 'stake': 4.0}]

In [70]:
exit_multiple = 3
last_valuation = company.rounds[-1].valuation 
total_shares = company.rounds[-1].shares
exit_valuation = last_valuation*exit_multiple

val_to_dist = exit_valuation
aux_investments = []
for i in company.investors:
    for a in i.investments:
        aux_investment = {
            'name': i.name,
            'round': a.round.name,
            'invested': a.invested,
            'shares': a.shares,
            'stake': a.shares/total_shares,
            'round_size': a.round.invested,
            'round_share': a.invested/a.round.invested,
            'liq_pref': a.round.covenants.liquidity_preference,
            'stake_value': a.shares/total_shares*exit_valuation,
            'liq_pref_floor':  a.invested * a.round.covenants.liquidity_preference
        }
        aux_investments.append(aux_investment)

total_no_preference_shares = 0 

for r in company.rounds[::-1]:
    r_invested = r.invested
    r_liq_pref = r.covenants.liquidity_preference
    r_to_pay = np.minimum(r_invested * r_liq_pref, val_to_dist)
    for k,i in enumerate(aux_investments):
        if i['round'] == r.name and r.covenants.liquidity_preference:
            stake_value = i['stake_value']
            liquidity_pref_floor = np.minimum(i['invested']*i['liq_pref'], r_to_pay * i['round_share'])
            total_paid = np.minimum(np.maximum(liquidity_pref_floor, stake_value), val_to_dist)
            aux_investments[k]['total_paid']  = total_paid
            aux_investments[k]['paid_due_to_stake'] = stake_value
            aux_investments[k]['paid_due_to_pref'] = aux_investments[k]['total_paid']-stake_value
            val_to_dist -= total_paid
        else:
            total_no_preference_shares += i['shares']

for r in company.rounds[::-1]:          
    for k,i in enumerate(aux_investments):
        if i['round'] == r.name and not r.covenants.liquidity_preference:
            aux_investments[k]['paid_due_to_stake'] = i['stake_value']
            aux_investments[k]['total_paid'] =  i['shares'] / total_no_preference_shares * val_to_dist
            aux_investments[k]['paid_due_to_pref'] = aux_investments[k]['total_paid']-i['stake_value']
            
print('---------------------------------------------')
print('last valuation: %s' % last_valuation)
print('exit valuation: %s' % exit_valuation)
print('---------------------------------------------')
print('name', '|', 'round', '|', 'invested', '|', 'stake', '|', 'total_paid')
print('stake_value', '|', 'liq_pref_floor', '|', 'paid_due_to_pref', '|', 'paid_due_to_stake')
print('------------------------------------------------------')
for i in aux_investments:
    print(i['name'], '|', i['round'], '|', i['invested'], '|', i['stake'], '|', i['total_paid'])
    print(i['stake_value'], '|', i['liq_pref_floor'], '|', i['paid_due_to_pref'], '|', i['paid_due_to_stake'])
    print('------------------------------------------------------')

---------------------------------------------
last valuation: 2500000.0
exit valuation: 7500000.0
---------------------------------------------
name | round | invested | stake | total_paid
stake_value | liq_pref_floor | paid_due_to_pref | paid_due_to_stake
------------------------------------------------------
founder | incorporation | 10000.0 | 0.72 | 1542857.142857143
5400000.0 | 0.0 | -3857142.8571428573 | 5400000.0
------------------------------------------------------
ze | seed | 10000.0 | 0.08000000000000002 | 171428.57142857148
600000.0000000001 | 0.0 | -428571.42857142864 | 600000.0000000001
------------------------------------------------------
tera | a | 400000.0 | 0.16 | 1200000.0
1200000.0 | 800000.0 | 0.0 | 1200000.0
------------------------------------------------------
patria | a | 100000.0 | 0.04 | 300000.0
300000.0 | 200000.0 | 0.0 | 300000.0
------------------------------------------------------


In [71]:
# import copy

# start = 1e4
# rounds = ['seed', 'a', 'b', 'c', 'd', 'exit']
# growth = [0, 0.1, 0.2, 0.5, 0.75, 0.8, 1, 2, 5, 7.5, 8, 10, 20, 50, 75, 80, 100]
# company_stake = np.linspace(1, 100, 18) #ajustar não faz sentido comprar 0% da companhia
# current_investor_deal_stake = np.linspace(0, 100, 21)
# liq_pref = np.linspace(0,3, 13)

# scenarios = []

# company.current_stake()
# company = Company('consorciei', capital, capital/price_per_share)
# for i in rounds:
#     for a in growth:
#         #exit or growth = 0 -> PnL (MOIC)
#         for b in company_stake:
#             #seed or 100% as an exit -> PnL (MOIC)
#             for c in current_investor_deal_stake:
#                 # novo investidor entra com o que sobrar, investidores antigos na proporção do stake atual sem considerar o founder
#                 for d in liq_pref:
#                     deal = Deal(500000, 20)
#                     deal.add_investor(BookInvestor('tera', 80))
#                     deal.add_investor(BookInvestor('patria', 20))
#                     covenants = Covenants({'liquidity_preference': 2})
#                     company.new_round('a', deal, covenants)



In [72]:
probabilidades = {'next_round': [0.5362, 0.3223, 0.1935,0.1130, 0.0613,0.0309],
                 'write_off': [0.1769, 0.0579, 0.0237,0.0095,0.0046,0.0021],
                 'ipo_or_nonr':[0.2869, 0.156, 0.105,0.0709, 0.0472,0.0283]}


In [461]:

capital = 1e4
price_per_share = 0.01

company = Company('consorciei', capital, capital/price_per_share)
company.incorporation.valuation

10000.0

In [338]:
cap = company.current_stake()
cap


[{'investor': 'founder', 'stake': 100.0}]

In [462]:
company.rounds[-1].valuation 

10000.0

In [463]:
deal = Deal(10000, 10)
deal.add_investor(BookInvestor('ze', 100))
company.new_round('seed', deal, Covenants({}))



deal = Deal(500000, 20)
deal.add_investor(BookInvestor('tera', 80))
deal.add_investor(BookInvestor('patria', 20))
covenants = Covenants({'liquidity_preference': 2})
company.new_round('a', deal, covenants)

company.rounds[-1].shares

1388888.888888889

In [464]:


capital = 1e4
price_per_share = 0.01

simulations = 20
number_of_rounds = 3


company = np.full((simulations), None, dtype=object)

for j in range (0,simulations):
    company[j] = Company(f'consorciei {j}', capital, capital/price_per_share)



In [465]:
deal = np.full((simulations, number_of_rounds), None, dtype=object)


for j in range (0,simulations):
    for i in range(0, number_of_rounds):
            deal[j][i] = Deal(random.choice(np.linspace(10000,100000,10)), random.choice(np.linspace(1,99,10)))
            deal[j][i].add_investor(BookInvestor(f'investidor_{i}', 100))
            company[j].new_round(f'round_{i}', deal[j][i], Covenants({'liquidity_preference': random.choice(np.linspace(0,2,5))}))



#cap = company.current_stake()

In [466]:
company[2].rounds[-1].valuation 

841121.4953271028

In [467]:
for i in range(0,simulations):
    print(company[i].current_stake())

[{'investor': 'founder', 'stake': 7.591833333333331}, {'investor': 'investidor_0', 'stake': 25.738166666666658}, {'investor': 'investidor_1', 'stake': 65.67}, {'investor': 'investidor_2', 'stake': 1.0}]
[{'investor': 'founder', 'stake': 11.583619341563784}, {'investor': 'investidor_0', 'stake': 3.4167510288065834}, {'investor': 'investidor_1', 'stake': 18.666296296296295}, {'investor': 'investidor_2', 'stake': 66.33333333333334}]
[{'investor': 'founder', 'stake': 3.5267288065843583}, {'investor': 'investidor_0', 'stake': 6.948703292181064}, {'investor': 'investidor_1', 'stake': 77.63567901234568}, {'investor': 'investidor_2', 'stake': 11.88888888888889}]
[{'investor': 'founder', 'stake': 12.502722222222221}, {'investor': 'investidor_0', 'stake': 42.387277777777776}, {'investor': 'investidor_1', 'stake': 44.10999999999999}, {'investor': 'investidor_2', 'stake': 1.0}]
[{'investor': 'founder', 'stake': 17.245438271604932}, {'investor': 'investidor_0', 'stake': 33.97863580246914}, {'invest

In [468]:

company[2].rounds[0].valuation


10000.0

In [469]:
company[2].rounds[1].valuation


75376.88442211054

In [470]:
company[2].rounds[2].valuation


113493.06431273643

In [471]:
company[2].rounds[3].valuation


841121.4953271028

In [472]:
exit_multiple = 3
last_valuation = company[2].rounds[-1].valuation 
total_shares = company[2].rounds[-1].shares
exit_valuation = last_valuation*exit_multiple

val_to_dist = exit_valuation
aux_investments = []
for i in company[2].investors:
    for a in i.investments:
        aux_investment = {
            'name': i.name,
            'round': a.round.name,
            'invested': a.invested,
            'shares': a.shares,
            'stake': a.shares/total_shares,
            'round_size': a.round.invested,
            'round_share': a.invested/a.round.invested,
            'liq_pref': a.round.covenants.liquidity_preference,
            'stake_value': a.shares/total_shares*exit_valuation,
            'liq_pref_floor':  a.invested * a.round.covenants.liquidity_preference
        }
        aux_investments.append(aux_investment)

total_no_preference_shares = 0 

for r in company[2].rounds[::-1]:
    r_invested = r.invested
    r_liq_pref = r.covenants.liquidity_preference
    r_to_pay = np.minimum(r_invested * r_liq_pref, val_to_dist)
    for k,i in enumerate(aux_investments):
        if i['round'] == r.name and r.covenants.liquidity_preference:
            stake_value = i['stake_value']
            liquidity_pref_floor = np.minimum(i['invested']*i['liq_pref'], r_to_pay * i['round_share'])
            total_paid = np.minimum(np.maximum(liquidity_pref_floor, stake_value), val_to_dist)
            aux_investments[k]['total_paid']  = total_paid
            aux_investments[k]['paid_due_to_stake'] = stake_value
            aux_investments[k]['paid_due_to_pref'] = aux_investments[k]['total_paid']-stake_value
            val_to_dist -= total_paid
        else:
            total_no_preference_shares += i['shares']

for r in company[2].rounds[::-1]:          
    for k,i in enumerate(aux_investments):
        if i['round'] == r.name and not r.covenants.liquidity_preference:
            aux_investments[k]['paid_due_to_stake'] = i['stake_value']
            aux_investments[k]['total_paid'] =  i['shares'] / total_no_preference_shares * val_to_dist
            aux_investments[k]['paid_due_to_pref'] = aux_investments[k]['total_paid']-i['stake_value']

res= pd.DataFrame()
            
print('---------------------------------------------')
print('last valuation: %s' % last_valuation)
print('exit valuation: %s' % exit_valuation)
print('---------------------------------------------')
print('name', '|', 'round', '|', 'invested', '|', 'stake', '|', 'total_paid')
print('stake_value', '|', 'liq_pref_floor', '|', 'paid_due_to_pref', '|', 'paid_due_to_stake')
print('------------------------------------------------------')
for i in aux_investments:
    print(i['name'], '|', i['round'], '|', i['invested'], '|', i['stake'], '|', i['total_paid'])
    print(i['stake_value'], '|', i['liq_pref_floor'], '|', i['paid_due_to_pref'], '|', i['paid_due_to_stake'])
    print('------------------------------------------------------')

---------------------------------------------
last valuation: 841121.4953271028
exit valuation: 2523364.4859813084
---------------------------------------------
name | round | invested | stake | total_paid
stake_value | liq_pref_floor | paid_due_to_pref | paid_due_to_stake
------------------------------------------------------
founder | incorporation | 10000.0 | 0.03526728806584358 | 1034.0158012016886
88992.22222222212 | 0.0 | -87958.20642102043 | 88992.22222222212
------------------------------------------------------
investidor_0 | round_0 | 50000.0 | 0.06948703292181065 | 175341.11111111098
175341.11111111098 | 75000.0 | 0.0 | 175341.11111111098
------------------------------------------------------
investidor_1 | round_1 | 100000.0 | 0.7763567901234568 | 1959031.1526479751
1959031.1526479751 | 200000.0 | 0.0 | 1959031.1526479751
------------------------------------------------------
investidor_2 | round_2 | 100000.0 | 0.11888888888888889 | 300000.0
300000.0 | 150000.0 | 0.0 | 3000

In [496]:

for i in aux_investments:
    print(f'{i["round"]}_paid_due_to_pref')
    print(f'{i["paid_due_to_pref"]}')



incorporation_paid_due_to_pref
-87958.20642102043
round_0_paid_due_to_pref
0.0
round_1_paid_due_to_pref
0.0
round_2_paid_due_to_pref
0.0


In [562]:
x= np.array([[i["paid_due_to_pref"]] for i in aux_investments]).T
x

array([[-87958.20642102,      0.        ,      0.        ,
             0.        ]])

In [578]:
df=pd.DataFrame(data=np.array([[i["invested"]] for i in aux_investments]).T,
                columns= [f'{i["round"]}_invested' for i in aux_investments])

df1=pd.DataFrame(data=np.array([[i["total_paid"]] for i in aux_investments]).T,
                columns= [f'{i["round"]}_total_paid' for i in aux_investments])

df2=pd.DataFrame(data=np.array([[i["liq_pref_floor"]] for i in aux_investments]).T,
                columns= [f'{i["round"]}_liq_pref_floor' for i in aux_investments])

df3=pd.DataFrame(data=np.array([[i["paid_due_to_pref"]] for i in aux_investments]).T,
                columns= [f'{i["round"]}_paid_due_to_pref' for i in aux_investments])

df4=pd.DataFrame(data=np.array([[i["paid_due_to_stake"]] for i in aux_investments]).T,
                columns= [f'{i["round"]}_paid_due_to_stake' for i in aux_investments])


 
df

SyntaxError: unmatched '}' (143828305.py, line 18)

In [567]:
df=pd.DataFrame(
                columns= [f'{i["round"]}_paid_due_to_pref' for i in aux_investments]) 
df

,incorporation_paid_due_to_pref,round_0_paid_due_to_pref,round_1_paid_due_to_pref,round_2_paid_due_to_pref


In [504]:
x= [i['round'] for i in aux_investments]



['incorporation', 'round_0', 'round_1', 'round_2']

In [415]:
res = {'last_valuation':last_valuation, 'exit_valuation': exit_valuation, 'round':[i for in aux_investments: 'round':i['round']]}
                                                                                                             # 'invested':i['invested'],
                                                                                                            # 'stake':i['stake'],
                                                                                                            # 'total_paid':i['total_paid'],
                                                                                                            # 'stake_value':i['stake_value']]}

SyntaxError: invalid syntax (3502863998.py, line 1)

In [440]:
x= for i f'{i['round']}'

SyntaxError: f-string: unmatched '[' (2545259538.py, line 1)

In [442]:
x= {'dict':1,'dict1':2, 'dict3': [
                                    [i["name"] for i in aux_investments],
                                    [i["round"] for i in aux_investments],
                                    [i["invested"] for i in aux_investments],
                                    [i["stake"] for i in aux_investments],
                                    [i["total_paid"] for i in aux_investments],
                                    [i["stake_value"] for i in aux_investments],
                                    [i["liq_pref_floor"] for i in aux_investments],
                                    [i["paid_due_to_pref"] for i in aux_investments],
                                    [i["paid_due_to_stake"] for i in aux_investments]]}


""
